# Machine Learning with PyTorch

## Tasks with Networks

<font size="+1">A simple feature classifier</font>
<a href="NetworkExamples_0.ipynb"><img src="img/open-notebook.png" align="right"/></a>

<font size="+1">An image classifier</font>
<a href="NetworkExamples_1.ipynb"><img src="img/open-notebook.png" align="right"/></a>

<font size="+1">A regression prediction</font>
<a href="NetworkExamples_2.ipynb"><img src="img/open-notebook.png" align="right"/></a>

<font size="+1">Clustering with PyTorch</font>
<a href="NetworkExamples_3.ipynb"><img src="img/open-notebook.png" align="right"/></a>

<font size="+1">Generative Adversarial Networks (GAN)</font> 
<a href="NetworkExamples_4.ipynb"><img src="img/open-notebook.png" align="right"/></a>

<font size="+1"><u><b>Part of Speech Tagger</b></u></font>
<a href="NetworkExamples_5.ipynb"><img src="img/open-notebook.png" align="right"/></a>

## Part of Speech Tagger

In this section we create an LSTM-based recurrent neural network to identify parts of speech.  This example is taken very closely from the official [PyTorch tutorial](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html#example-an-lstm-for-part-of-speech-tagging) by Robert Guthrie.  Some minor changes are made to the code and new commentary is introduced.

The example we will look at here is enhanced and reworked and enhanced in an official AllenNLP tutorial that is also presented in the next chapter.  Comparing the styles of these two notebooks is a useful exercise.  The example here uses a very small toy training set; however, the same network *could* be trained against a robust corpus and produce reasonably good classification.

### Remembering state in a network

As was mentioned briefly, the class of network layers called Recurrent Neural Networks are able to remember state within a layer. In the case of an LSTM (Long Short-term Memory) layer, there are extra tensors defining the *hidden state* along with the direct state of the weights for that layer.  The hidden state is able to remember information from arbitrary earlier states.

In particular, an LSTM has three additional activation functions for the hidden state.  One activation function (also called a "gate") is the *input gate*, the next is the "forget gate", the last is the "output gate."  Each of these gates is parameterized differently, and they each take the same inputs as would a fully-connected (linear) layer.  The key difference is that *some but not all* of the current hidden state will contribute to the input gate.

Whether a particular neuron is involved in feedback is based simply on the activation of its output gate.  But as well, depending on the parameters to the forget gate and the inputs, that hidden neuron may become free to be retrained on new information.  An illustration at the [A.I. Wiki](https://skymind.ai/wiki/lstm#long) provides a good overview.

![LSTM gating](img/gers_lstm.png)

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [2]:
lstm = nn.LSTM(3, 3)  # Input dim is 3, output dim is 3
inputs = [torch.randn(1, 3) for _ in range(5)]  # make a sequence of length 5

# initialize the hidden state.
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))

for i in inputs:
    # Step through the sequence one element at a time.
    # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)

# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>)
(tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>), tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<StackBackward>))


In [10]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    ("the dog ate the apple".split(), ["DET", "NOUN", "VERB", "DET", "NOUN"]),
    ("we often ate pie".split(), ["PRON", "PART", "VERB", "NOUN"]),
    ("everybody read that book".split(), ["NOUN", "VERB", "DET", "NOUN"]),
    ("do not dog me".split(), ["VERB", "PART", "VERB", "PRON"]),
    ("do not dog the dog".split(), ["VERB", "PART", "VERB", "DET", "NOUN"]),
    ("we dog everybody".split(), ["PRON", "VERB", "NOUN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
            
print(word_to_ix)

{'the': 0, 'dog': 1, 'ate': 2, 'apple': 3, 'we': 4, 'often': 5, 'pie': 6, 'everybody': 7, 'read': 8, 'that': 9, 'book': 10, 'do': 11, 'not': 12, 'me': 13}


In [11]:
tag_to_ix = {"DET": 0, "NOUN": 1, "VERB": 2, "PRON": 3, "PART": 4}
ix_to_tag = {v:k for (k, v) in tag_to_ix.items()}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

In [12]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [13]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print('Initial')
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

Initial
tensor([[-1.3415, -1.4094, -1.4865, -2.0375, -1.9824],
        [-1.3193, -1.4269, -1.6062, -1.9190, -1.9295],
        [-1.2652, -1.4732, -1.6676, -1.9118, -1.8832],
        [-1.3094, -1.3948, -1.6339, -1.9272, -1.9561],
        [-1.3424, -1.4249, -1.5009, -1.9967, -1.9693]])


In [14]:
# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print("Trained")
    print(tag_scores)

Trained
tensor([[-0.0196, -4.4740, -7.2709, -5.2879, -6.0837],
        [-5.2160, -0.0580, -2.9891, -8.7630, -7.8053],
        [-6.8094, -4.1231, -0.0253, -4.9788, -7.1846],
        [-0.0213, -4.3791, -6.4513, -4.9825, -9.5001],
        [-4.3551, -0.0480, -3.4080, -8.7591, -7.1231]])


In [16]:
test_data = [
    "the dog read the book".split(),
    "we dog the dog".split()
]

for sentence in test_data:
    inputs = prepare_sequence(sentence, word_to_ix)
    for word, scores in zip(sentence, model(inputs)):
        best = torch.argmax(scores).item()
        part = ix_to_tag[best]
        print(f"{word}[{part}]", end=' ')
    print()

the[DET] dog[NOUN] read[VERB] the[DET] book[NOUN] 
we[PRON] dog[VERB] the[DET] dog[NOUN] 


## Next Lesson

**Natural Language Processing**: This lesson looked at Reinforcement Learning.  In the next section we turn to the AllenNLP extension to PyTorch.

<a href="AllenNLP_0.ipynb"><img src="img/open-notebook.png" align="left"/></a>